In [ ]:

#import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.applications import ResNet152,VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
#from google.colab.patches import cv2_imshow
#from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2 as cv 
import os
import json
import pickle
import tensorflow as tf
from tensorflow import keras

In [ ]:
path="/content/drive/MyDrive/Dataset/UCF/ucf/"

In [ ]:
label_list=[]
for file in os.listdir(path):
    filename= os.fsdecode(path+file)
    #print("Filename: ",filename)
    name=filename.split('/')[7]
    #print("Name: ", name)
    label_list.append(name)  
label={}#
idx=0
for i in label_list:
    label[i]=idx
    idx+=1
for i in label_list:
    print(i)
    print(label[i])

In [ ]:

label=[]
image=[]

In [ ]:

def optical_image(folder_name):

    curr_folder=os.fsdecode(path+folder_name)
    file_name=folder_name+".data"
    count2=0
    for file1 in os.listdir(curr_folder):
        if count2==1:
            break
        if file1.endswith('.avi'):
            print(file1)
            
            cap = cv.VideoCapture(curr_folder+'/' + file1)
            #print("cap: ",cap.isOpened())
            ret, first_frame = cap.read()
            prev_gray = cv.cvtColor(first_frame, cv.COLOR_BGR2GRAY)
            mask = np.zeros_like(first_frame)
            
            mask[..., 1] = 255
            count=0
            while(cap.isOpened()):

                ret, frame = cap.read()
                if (ret != True):
                    break
                gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
                flow = cv.calcOpticalFlowFarneback(prev_gray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                magnitude, angle = cv.cartToPolar(flow[..., 0], flow[..., 1])
                mask[..., 0] = angle * 180 / np.pi / 2
                mask[..., 2] = cv.normalize(magnitude, None, 0, 255, cv.NORM_MINMAX)
                rgb = cv.cvtColor(mask, cv.COLOR_HSV2BGR)
             
               
                rgb = cv.resize(rgb, (224, 224))
                #cv.imshow("dense optical flow", rgb)
                #Here rgb is optical flow of one frame
                rgb=np.array(rgb)
            
                # Updates previous frame
                prev_gray = gray
                if cv.waitKey(1) & 0xFF == ord('q'):
                     break
           
                
                #store all video's OP_Flow of each category(ApplyMakeUp) in images
                image.append(rgb)

                #mapping each video's OP_flow with the corresponding category name
                label.append(folder_name)
               
                count=count+1
            count2=count2+1
            cap.release()
            cv.destroyAllWindows()    
    
    

In [ ]:
#creating list of each folder name and store in folder_list
folder_list=[]

dirs=os.listdir(path)

for file in dirs:
    filename=os.fsdecode(path+file)
    #print(filename)
    name=filename.split('/')[7]
   # optical_image2(name)
    print(name)
    folder_list.append(name)

In [ ]:
label_mapping=[]#mapping or labeling each frame with the video folder_name
_allShapeLable=np.zeros(())
for item in folder_list:
    print("----la la-----: ", item)
   
    optical_image(item)

In [ ]:

with open("/content/drive/MyDrive/Dataset/laa.data", 'wb') as f:
    #pickle.dump(label, f)
        
with open("/content/drive/MyDrive/Dataset/imm.data", 'wb') as f:
#pickle.dump(image, f)        

In [ ]:
with open("/content/drive/MyDrive/Dataset/imm.data",'rb') as f:
    images=pickle.load( f)
image = np.asarray(images) 
print(image.shape)
with open("/content/drive/MyDrive/Dataset/laa.data", 'rb') as l:
    labels=pickle.load(l)
label= np.asarray(labels) 
print(label.shape)

In [ ]:
label= np.asarray(label)
lb = LabelBinarizer()
labels = lb.fit_transform(label)
print(labels.shape)

In [ ]:
image = np.asarray(image) 
print(image.shape)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(image, labels,
    test_size=0.25, stratify=labels, random_state=42)

In [ ]:
print(trainX.shape,testX.shape)

In [ ]:
trainAug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")
#valAug = ImageDataGenerator()
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
#valAug.mean = mean

In [ ]:
# load the ResNet-50 network, ensuring the head FC layer sets are left
# off
baseModel = ResNet152(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224,224, 3)))
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(lb.classes_), activation="softmax")(headModel)
model = Model(inputs=baseModel.input, outputs=headModel)

for layer in baseModel.layers:
	layer.trainable = False

In [ ]:
from keras.callbacks import ModelCheckpoint
save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# layers to being non-trainable)
print("[INFO] compiling model...")
#opt = SGD(lr=1e-4, momentum=0.9, decay=1e-4 / 70)
model.compile(loss="categorical_crossentropy", optimizer="rmsprop",
	metrics=["accuracy"])
# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(
 x=trainAug.flow(trainX, trainY), batch_size=64,

 validation_data=(testX, testY),
   
    epochs=120)

In [ ]:
import keras
from matplotlib import pyplot as plt

plt.plot(H.history['accuracy'])
plt.plot(H.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import keras
from matplotlib import pyplot as plt

plt.plot(H.history['loss'])
plt.plot(H.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss, acc = model.evaluate(testX ,testY, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))